In [96]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
from torch import nn


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [97]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])


train_data_path = "C:\\large_files\\data_weather\\train"
test_data_path = "C:\\large_files\\data_weather\\val"

BATCH_SIZE = 16
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)

test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE)

In [98]:
classes = train_dataset.classes

class_idx = {}

for i in range(4):
    class_idx[i] = classes[i]

class_idx

{0: 'cloudy', 1: 'rainy', 2: 'shiny', 3: 'sunrise'}

In [99]:
data, _ = next(iter(train_dataloader))

data.shape

torch.Size([16, 3, 64, 64])

In [100]:
class weatherClassificationModelv0(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(in_features=32*16*16, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=4)
        self.relu = nn.ReLU()

    def forward(self, x : torch.tensor)-> torch.Tensor:
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32*16*16)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [101]:
modelv0 = weatherClassificationModelv0()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelv0.parameters(), lr=0.001)

In [102]:
def accuracy_function(y_true, y_pred):

    correct = torch.eq(y_true, y_pred).sum()
    acc = (correct/ len(y_pred))*100
    return acc

In [103]:
def train(model : torch.nn.Module, data_loader : torch.utils.data.DataLoader, loss_fn : torch.nn.Module, optimizer : torch.optim.Optimizer, accuracy_function, device : torch.device = device):

    model.to(device)
    train_loss = 0
    acc = 0
    model.train()
    for batch, (img, label) in enumerate(data_loader):

        img, label = img.to(device), label.to(device)
        train_preds = model(img)
        loss = loss_fn(train_preds, label)
        train_loss+=loss
        acc+=accuracy_function(y_true = label, y_pred = train_preds.argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if batch % 53 == 0:
            print(f"Looked at {(batch+1) * len(img)}/{len(train_dataloader.dataset)} samples.")

    train_loss /= len(data_loader)
    acc /= len(data_loader)

    print(f"Train loss : {train_loss:.4f}, train acc : {acc:2f}%")

In [104]:
def test(model : torch.nn.Module,data_loader : torch.utils.data.DataLoader, loss_fn : torch.nn.Module, accuracy_function, device : torch.device = device):

    model.to(device)
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():

        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            test_preds = model(x)
            test_loss += loss_fn(test_preds, y)
            test_acc += accuracy_function(y_true = y, y_pred = test_preds.argmax(dim=1))

        test_loss /= len(data_loader)
        test_acc /= len(data_loader)

        print(f"Test loss : {test_loss:.3f}, test acc : {test_acc:.2f}%")

In [105]:
epochs = 20

for epoch in range(epochs):
    print(f"Epoch : {epoch+1}")

    train(model=modelv0, data_loader=train_dataloader, loss_fn = loss_fn, optimizer = optimizer, accuracy_function=accuracy_function, device = device)

    test(model=modelv0, data_loader=test_dataloader, loss_fn=loss_fn, accuracy_function=accuracy_function, device = device)

Epoch : 1
Looked at 0/907 samples.
Looked at 848/907 samples.
Train loss : 0.8718, train acc : 62.789074%
Test loss : 0.642, test acc : 78.04%
Epoch : 2
Looked at 0/907 samples.
Looked at 848/907 samples.
Train loss : 0.5415, train acc : 78.568581%
Test loss : 0.498, test acc : 79.91%
Epoch : 3
Looked at 0/907 samples.
Looked at 848/907 samples.
Train loss : 0.4746, train acc : 82.137161%
Test loss : 0.502, test acc : 83.48%
Epoch : 4
Looked at 0/907 samples.
Looked at 848/907 samples.
Train loss : 0.4093, train acc : 83.941391%
Test loss : 0.413, test acc : 84.38%
Epoch : 5
Looked at 0/907 samples.
Looked at 848/907 samples.
Train loss : 0.3708, train acc : 85.755585%
Test loss : 0.324, test acc : 87.50%
Epoch : 6
Looked at 0/907 samples.
Looked at 848/907 samples.
Train loss : 0.3080, train acc : 88.985252%
Test loss : 0.306, test acc : 89.73%
Epoch : 7
Looked at 0/907 samples.
Looked at 848/907 samples.
Train loss : 0.2705, train acc : 89.374008%
Test loss : 0.378, test acc : 86.16%

In [123]:
from PIL import Image

img_path = "C:\\large_files\\data_weather\\val\\rainy\\rain22.jpeg"

img = Image.open(img_path)

img = transform(img).unsqueeze(0)

modelv0.eval()
with torch.no_grad():
    pred = modelv0(img)

    print(class_idx[pred.argmax(dim=1).item()])

rainy


In [126]:
model_path = "../models/weather_classification_modelv0.pth"
torch.save(modelv0.state_dict(), model_path)